# Mean Reversion Trading Strategy Demo 📈

## Interactive demonstration of the Ornstein-Uhlenbeck mean reversion strategy

This notebook walks you through:
1. **Data Collection** - Fetching real market data
2. **Parameter Estimation** - Using Maximum Likelihood Estimation
3. **Signal Generation** - Creating buy/sell signals
4. **Backtesting** - Evaluating strategy performance
5. **Visualization** - Interactive plots and analysis

## 🚀 Setup and Imports

In [ ]:
# Install required packages if running in Colab
import sys
if 'google.colab' in sys.modules:
    !pip install yfinance

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All imports successful!")
print("🚀 Ready to start the demo!")

## 📊 Step 1: Data Collection

Let's fetch real market data for a stock. You can change the ticker and date range below:

In [ ]:
# Configuration
TICKER = "AAPL"  # Change this to any ticker you want
START_DATE = "2022-01-01"
END_DATE = "2024-01-01"

print(f"📈 Fetching data for {TICKER} from {START_DATE} to {END_DATE}...")

# Fetch data
data = yf.download(TICKER, start=START_DATE, end=END_DATE)
prices = data['Close'].values

print(f"✅ Successfully fetched {len(prices)} data points")
print(f"📊 Price range: ${prices.min():.2f} - ${prices.max():.2f}")

# Plot the raw data
plt.figure(figsize=(14, 6))
plt.plot(data.index, prices, linewidth=1.5, alpha=0.8)
plt.title(f'{TICKER} Stock Price Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 🔢 Step 2: Parameter Estimation Using Maximum Likelihood

Now we'll estimate the Ornstein-Uhlenbeck process parameters:
- **μ (mu)**: Long-term mean
- **θ (theta)**: Speed of mean reversion
- **σ (sigma)**: Volatility

In [ ]:
def estimate_ou_parameters(prices):
    """
    Estimate Ornstein-Uhlenbeck parameters using Maximum Likelihood Estimation
    """
    n = len(prices)
    dt = 1 / 252  # Daily data, 252 trading days per year
    
    def ou_log_likelihood(params):
        mu, theta, sigma = params
        
        if theta <= 0 or sigma <= 0:
            return 1e6
            
        X_diff = np.diff(prices)
        X_lag = prices[:-1]
        
        # Expected change according to OU process
        expected_diff = theta * (mu - X_lag) * dt
        
        # Residuals
        residuals = X_diff - expected_diff
        
        # Log likelihood
        variance = sigma**2 * dt
        log_likelihood = -0.5 * len(residuals) * np.log(2 * np.pi * variance) - \
                        0.5 * np.sum(residuals**2) / variance
                        
        return -log_likelihood  # Return negative for minimization
    
    # Initial parameter guess
    mu_init = np.mean(prices)
    theta_init = 0.1
    sigma_init = np.std(np.diff(prices)) / np.sqrt(dt)
    
    initial_params = [mu_init, theta_init, sigma_init]
    
    # Optimize
    result = minimize(ou_log_likelihood, initial_params, 
                     method='L-BFGS-B', 
                     bounds=[(None, None), (1e-6, None), (1e-6, None)])
    
    return result.x

# Estimate parameters
print("🔍 Estimating Ornstein-Uhlenbeck parameters...")
mu_est, theta_est, sigma_est = estimate_ou_parameters(prices)

print(f"\n📊 Estimated Parameters:")
print(f"   μ (Long-term mean): ${mu_est:.2f}")
print(f"   θ (Mean reversion speed): {theta_est:.4f}")
print(f"   σ (Volatility): ${sigma_est:.2f}")
print(f"\n💡 Interpretation:")
print(f"   - The stock tends to revert to ${mu_est:.2f}")
print(f"   - Half-life of mean reversion: {np.log(2)/theta_est:.1f} days")
print(f"   - Daily volatility: ${sigma_est * np.sqrt(1/252):.2f}")

## 📈 Step 3: Generate Trading Signals

Based on the estimated parameters, we'll generate buy and sell signals:

In [ ]:
def generate_signals(prices, mu, sigma, threshold_multiplier=1.5):
    """
    Generate buy and sell signals based on deviation from mean
    """
    threshold = threshold_multiplier * sigma * np.sqrt(1/252)  # Daily threshold
    
    buy_signals = prices < (mu - threshold)
    sell_signals = prices > (mu + threshold)
    
    return buy_signals, sell_signals, threshold

# Generate signals
buy_signals, sell_signals, threshold = generate_signals(prices, mu_est, sigma_est)

print(f"📊 Signal Summary:")
print(f"   Buy signals: {np.sum(buy_signals)} ({np.sum(buy_signals)/len(prices)*100:.1f}% of days)")
print(f"   Sell signals: {np.sum(sell_signals)} ({np.sum(sell_signals)/len(prices)*100:.1f}% of days)")
print(f"   Threshold: ${threshold:.2f}")

# Plot signals
plt.figure(figsize=(14, 8))
plt.plot(data.index, prices, label='Price', linewidth=1.5, alpha=0.8)
plt.axhline(mu_est, color='black', linestyle='--', label=f'Mean (${mu_est:.2f})', alpha=0.7)
plt.axhline(mu_est + threshold, color='red', linestyle=':', label=f'Sell Threshold', alpha=0.7)
plt.axhline(mu_est - threshold, color='green', linestyle=':', label=f'Buy Threshold', alpha=0.7)

# Plot signals
buy_dates = data.index[buy_signals]
sell_dates = data.index[sell_signals]

if len(buy_dates) > 0:
    plt.scatter(buy_dates, prices[buy_signals], color='green', marker='^', 
               s=50, label=f'Buy Signals ({len(buy_dates)})', zorder=5)
               
if len(sell_dates) > 0:
    plt.scatter(sell_dates, prices[sell_signals], color='red', marker='v', 
               s=50, label=f'Sell Signals ({len(sell_dates)})', zorder=5)

plt.title(f'{TICKER} - Mean Reversion Trading Signals', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 🎯 Step 4: Backtest the Strategy

Now let's see how our strategy would have performed:

In [ ]:
def backtest_strategy(prices, buy_signals, sell_signals, initial_capital=10000):
    """
    Backtest the mean reversion strategy
    """
    portfolio_value = np.zeros(len(prices))
    cash = initial_capital
    position = 0  # 0: no position, 1: long, -1: short
    shares = 0
    trades = []
    
    portfolio_value[0] = initial_capital
    
    for i in range(1, len(prices)):
        current_price = prices[i]
        
        # Check for entry signals
        if position == 0:
            if buy_signals[i]:  # Enter long position
                shares = cash / current_price
                position = 1
                trades.append(('BUY', data.index[i], current_price, shares))
                cash = 0
            elif sell_signals[i]:  # Enter short position (simplified)
                position = -1
                trades.append(('SELL_SHORT', data.index[i], current_price, cash/current_price))
        
        # Check for exit signals
        elif position == 1 and sell_signals[i]:  # Exit long
            cash = shares * current_price
            trades.append(('SELL', data.index[i], current_price, shares))
            shares = 0
            position = 0
            
        elif position == -1 and buy_signals[i]:  # Exit short
            position = 0
            trades.append(('COVER', data.index[i], current_price, 0))
        
        # Calculate portfolio value
        if position == 1:
            portfolio_value[i] = shares * current_price
        else:
            portfolio_value[i] = cash
    
    return portfolio_value, trades

# Run backtest
print("🔄 Running backtest...")
portfolio_values, trades = backtest_strategy(prices, buy_signals, sell_signals)

# Calculate performance metrics
initial_capital = 10000
final_value = portfolio_values[-1]
total_return = (final_value - initial_capital) / initial_capital * 100
buy_hold_return = (prices[-1] - prices[0]) / prices[0] * 100

print(f"\n📊 Backtest Results:")
print(f"   Initial Capital: ${initial_capital:,.2f}")
print(f"   Final Portfolio Value: ${final_value:,.2f}")
print(f"   Total Return: {total_return:.2f}%")
print(f"   Buy & Hold Return: {buy_hold_return:.2f}%")
print(f"   Strategy vs Buy & Hold: {total_return - buy_hold_return:+.2f}%")
print(f"   Total Trades: {len(trades)}")

## 📊 Step 5: Visualization and Analysis

Let's create comprehensive visualizations:

In [ ]:
# Create comprehensive performance visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Price and signals
ax1.plot(data.index, prices, label='Price', alpha=0.8)
ax1.axhline(mu_est, color='black', linestyle='--', alpha=0.7, label='Mean')
if np.sum(buy_signals) > 0:
    ax1.scatter(data.index[buy_signals], prices[buy_signals], 
               color='green', marker='^', s=30, label='Buy')
if np.sum(sell_signals) > 0:
    ax1.scatter(data.index[sell_signals], prices[sell_signals], 
               color='red', marker='v', s=30, label='Sell')
ax1.set_title('Price and Trading Signals', fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Portfolio performance
buy_hold_portfolio = initial_capital * (prices / prices[0])
ax2.plot(data.index, portfolio_values, label='Strategy', linewidth=2)
ax2.plot(data.index, buy_hold_portfolio, label='Buy & Hold', alpha=0.7)
ax2.set_title('Portfolio Performance Comparison', fontweight='bold')
ax2.set_ylabel('Portfolio Value ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Price distribution
ax3.hist(prices, bins=50, alpha=0.7, density=True, color='skyblue')
ax3.axvline(mu_est, color='red', linestyle='--', linewidth=2, label=f'Estimated Mean: ${mu_est:.2f}')
ax3.axvline(np.mean(prices), color='green', linestyle=':', linewidth=2, label=f'Sample Mean: ${np.mean(prices):.2f}')
ax3.set_title('Price Distribution', fontweight='bold')
ax3.set_xlabel('Price ($)')
ax3.set_ylabel('Density')
ax3.legend()

# 4. Rolling returns
strategy_returns = np.diff(portfolio_values) / portfolio_values[:-1] * 100
buy_hold_returns = np.diff(buy_hold_portfolio) / buy_hold_portfolio[:-1] * 100

window = 30  # 30-day rolling window
strategy_rolling = pd.Series(strategy_returns).rolling(window).mean()
buy_hold_rolling = pd.Series(buy_hold_returns).rolling(window).mean()

ax4.plot(data.index[1:], strategy_rolling, label='Strategy (30-day avg)', linewidth=2)
ax4.plot(data.index[1:], buy_hold_rolling, label='Buy & Hold (30-day avg)', alpha=0.7)
ax4.axhline(0, color='black', linestyle='-', alpha=0.3)
ax4.set_title('Rolling Average Daily Returns', fontweight='bold')
ax4.set_xlabel('Date')
ax4.set_ylabel('Daily Return (%)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📋 Summary and Next Steps

Let's create a comprehensive summary:

In [ ]:
# Calculate additional metrics
strategy_returns = np.diff(portfolio_values) / portfolio_values[:-1]
strategy_volatility = np.std(strategy_returns) * np.sqrt(252)  # Annualized
strategy_sharpe = (total_return/100) / (strategy_volatility) if strategy_volatility > 0 else 0

buy_hold_returns_calc = np.diff(buy_hold_portfolio) / buy_hold_portfolio[:-1]
buy_hold_volatility = np.std(buy_hold_returns_calc) * np.sqrt(252)
buy_hold_sharpe = (buy_hold_return/100) / (buy_hold_volatility) if buy_hold_volatility > 0 else 0

print("="*60)
print(f"📊 COMPREHENSIVE STRATEGY ANALYSIS - {TICKER}")
print("="*60)
print(f"📅 Period: {START_DATE} to {END_DATE}")
print(f"📈 Trading Days: {len(prices)}")
print()
print("🔢 ORNSTEIN-UHLENBECK PARAMETERS:")
print(f"   μ (Long-term mean): ${mu_est:.2f}")
print(f"   θ (Mean reversion speed): {theta_est:.4f}")
print(f"   σ (Volatility): ${sigma_est:.2f}")
print(f"   Half-life: {np.log(2)/theta_est:.1f} days")
print()
print("📊 TRADING ACTIVITY:")
print(f"   Buy signals: {np.sum(buy_signals)} ({np.sum(buy_signals)/len(prices)*100:.1f}% of days)")
print(f"   Sell signals: {np.sum(sell_signals)} ({np.sum(sell_signals)/len(prices)*100:.1f}% of days)")
print(f"   Total trades: {len(trades)}")
print()
print("💰 PERFORMANCE COMPARISON:")
print(f"   Strategy Return: {total_return:+.2f}%")
print(f"   Buy & Hold Return: {buy_hold_return:+.2f}%")
print(f"   Excess Return: {total_return - buy_hold_return:+.2f}%")
print()
print("⚡ RISK METRICS:")
print(f"   Strategy Volatility: {strategy_volatility:.2f}")
print(f"   Buy & Hold Volatility: {buy_hold_volatility:.2f}")
print(f"   Strategy Sharpe Ratio: {strategy_sharpe:.2f}")
print(f"   Buy & Hold Sharpe Ratio: {buy_hold_sharpe:.2f}")
print()
print("✅ STRATEGY ASSESSMENT:")
if total_return > buy_hold_return:
    print("   🎯 Strategy OUTPERFORMED buy & hold")
else:
    print("   📉 Strategy UNDERPERFORMED buy & hold")
    
if strategy_sharpe > buy_hold_sharpe:
    print("   🏆 Strategy has BETTER risk-adjusted returns")
else:
    print("   ⚠️  Strategy has WORSE risk-adjusted returns")
    
print("="*60)
print("⚠️  DISCLAIMER: This is for educational purposes only.")
print("   Past performance does not guarantee future results.")
print("="*60)

## 🎮 Try Different Parameters!

Want to experiment? Go back and change:
- **TICKER**: Try different stocks (MSFT, GOOGL, TSLA, etc.)
- **Date ranges**: Different time periods
- **threshold_multiplier**: More or less sensitive signals

## 🚀 Next Steps

1. **Improve the strategy**: Add transaction costs, better risk management
2. **Test on more assets**: Portfolio of mean-reverting stocks
3. **Real-time implementation**: Connect to live data feeds
4. **Machine learning**: Use ML to optimize parameters

## 📚 Learn More

- [Ornstein-Uhlenbeck Process Theory](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process)
- [Mean Reversion in Finance](https://www.investopedia.com/terms/m/meanreversion.asp)
- [Quantitative Trading Strategies](https://www.quantstart.com/)

---

**🎉 Congratulations! You've successfully implemented and backtested a mean reversion trading strategy!**